# Computer languages

## Assembly language

All computer code is ultimately executed in 'assembly' language. These are the instructions that the CPU is designed to understand. The instructions are exectuted sequentially from a specified point in memory, and generally modify and move other parts of the computer's memory. Here is an example of 'Hello World' in assembly (from http://cs.lmu.edu/~ray/notes/gasexamples/):

            .global _start

            .text
    _start:
            # write(1, message, 13)
            mov     $1, %rax                # system call 1 is write
        mov     $1, %rdi                # file handle 1 is stdout
            mov     $message, %rsi          # address of string to output
        mov     $13, %rdx               # number of bytes
            syscall                         # invoke operating system to do the write

            # exit(0)
            mov     $60, %rax               # system call 60 is exit
            xor     %rdi, %rdi              # we want return code 0
            syscall                         # invoke operating system to exit
    message:
            .ascii  "Hello, world\n"


This code must be compiled so that the generic memory references (e.g., %rax) are converted to actual numbers. This is complicated, difficult to read, and nobody wants to do this ever. The only reason to program in assembly language is to play [CoreWars](http://www.corewars.org).


## Compiled languages

Most scientific programming has traditionally been done in 'low level', compiled languages like [FORTRAN](https://en.wikipedia.org/wiki/Fortran), occassionally [C](https://en.wikipedia.org/wiki/C_(programming_language)). FORTRAN was developed in 1956, but the cannonical FORTRAN used in most scientific applications was developed in 1977, called FORTRAN 77; it is still in wide use today. Most 'high level' languages build off of C and FORTRAN libraries developed decades ago, like [LAPACK](http://www.netlib.org/lapack/). FORTRAN is easy to read, but must be compiled, and may be complicated for things other than straightforward mathematical array opperations. This example shows code used to solve a laplace transform:

          subroutine timestep(u,n,m,dx,dy,error)
          
          double precision u(n,m)
          double precision dx,dy,dx2,dy2,dnr_inv,tmp,diff
          integer n,m,i,j

          dx2 = dx*dx
          dy2 = dy*dy
          dnr_inv = 0.5d0 / (dx2+dy2)
          error = 0
          do 200,j=2,m-1
             do 100,i=2,n-1
                tmp = u(i,j)
                u(i,j) = ((u(i-1,j) + u(i+1,j))*dy2+
         &           (u(i,j-1) + u(i,j+1))*dx2)*dnr_inv
                diff = u(i,j) - tmp
                error = error + diff*diff
     100     continue
     200  continue
          error = sqrt(error)
          end


## Interpreted languages

Most modern scientific analysis is done using 'high level' interpreted code. This type of language does not need to be compiled, but is rather interpreted (i.e., 'compiled' and converted to machine language) line by line. This allows users to either type code in interactively, or create a longer script or program that can be run all at once. The advantages of these high level languages are readability, flexibility, and compactness. For example, the nested loop above can be done in Python with:

    u[1:-1, 1:-1] = ((u[0:-2, 1:-1]+u[2:, 1:-1])*dy2+
        (u[1:-1, 0:-2]+u[1:-1, 2:])*dx2)*0.5/(dx2 + dy2)

There are also packages (in this case `scipy.ndimage.filters.laplace`) that will do the same operation as a function call. The primary disadvantage is that these languages are slow; most complicated programs that require speed (such as numerical weather prediction) still rely on FORTRAN. Generally, FORTRAN is about ten times faster than even optimized numerical python scripts (see [this comparison](http://scipy.github.io/old-wiki/pages/PerformancePython) of different methods used in speeding up pure python code.)


## Special considerations for mathematics on computers

All code will eventually be translated to machine language, commands that can be understood by the CPU. Computers are fundamentally digital, and represent all things using 32- 64- or 128- bit numbers. For example, a 32 bit representation of an integer is:

     42 = 00000000 00101010
    
Obviously, all integers can be represented exactly using binary numbers, but floating point numbers (numbers with decimals) cannot. These numbers need to be approximated. The standard way to do this is the [IEEE 754](https://en.wikipedia.org/wiki/IEEE_floating_point) floating point convention. Thus numbers are not stored exactly, but have a precision based on the number of bits used to represent the number. Typical precisions are 32 bit (single precision) and 64 bit (double precision). These numbers are computed as sign \* $2^{exponent}$ \* mantissa.

### Exercise:

Use this [converter](http://www.h-schmidt.net/FloatConverter/IEEE754.html) to look at representations of different numbers.

Note that you generally don't need to worry about the details of the representation of floating point numbers other than to remember that mathematical operations may have small errors that can compount over many iterations in an analysis. These errors are called 'roundoff errors'.

In [1]:
import numpy as np

# create a list of a million integers, 0 through 999999 
integers = np.arange(1000000)
# same thing, but with 32- and 64-bit floating point numbers
float32s = np.arange(1000000.0, dtype='float32')
float64s = np.arange(1000000.0, dtype='float64')

print(np.sum(integers)/1e6)
print(np.sum(float32s)/1e6)
print(np.sum(float64s)/1e6)

499999.5
499999.801344
499999.5


### The main consequence of roundoff errors are that you should:
  * Always use double precision numbers
  * Always compare numbers by seeing if they are close (i.e., the difference is smaller than some specified small value)